In [1]:
import torch

from torch_geometric.utils import to_dense_adj
from torch.utils.data import DataLoader, random_split

In [2]:
from helpers import CVFConfigDataset

In [3]:
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=False)

    def forward(self, x, adjacency):
        x = self.linear(x)
        # x = torch.sparse.mm(adjacency, x)
        x = torch.matmul(adjacency, x)
        return x

In [ ]:
dataset = CVFConfigDataset(
    "small_graph_test_config_rank_dataset.csv", "small_graph_edge_index.json", 4
)
# dataset = CVFConfigDataset(
#     "graph_1_config_rank_dataset.csv", "graph_1_edge_index.json", 10
# )
# dataset = CVFConfigDataset(
#     "graph_4_config_rank_dataset.csv", "graph_4_edge_index.json", 10
# )
# dataset = CVFConfigDataset(
#     "graph_5_config_rank_dataset.csv", "graph_5_edge_index.json", 10
# )
train_size = int(0.75 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [5]:
adjacency = to_dense_adj(dataset.edge_index.t().contiguous())[0]
adjacency += torch.eye(len(adjacency))
adjacency = adjacency.unsqueeze(0)
adjacency

tensor([[[1., 1., 1., 1.],
         [1., 1., 0., 0.],
         [1., 0., 1., 0.],
         [1., 0., 0., 1.]]])

In [6]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gnn1 = VanillaGNNLayer(dim_in, dim_h)
        self.gnn2 = VanillaGNNLayer(dim_h, dim_out)
        # self.out = torch.nn.Linear(dim_h, dim_out)

    def forward(self, x, adjacency):
        h = self.gnn1(x, adjacency)
        h = torch.relu(h)
        h = self.gnn2(h, adjacency)
        # h = torch.sigmoid(h)
        h = h.squeeze(-1)
        return h

    def fit(self, data_loader, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        self.train()
        for epoch in range(1, epochs + 1):
            avg_loss = 0
            count = 0
            for batch in data_loader:
                x = batch[0]
                print(x)
                y = batch[1]
                optimizer.zero_grad()
                out = self(x, adjacency)
                # print("output", out, "y", y)
                loss = criterion(out, y)
                avg_loss += loss
                count += 1
                loss.backward()
                optimizer.step()

            print("Loss:", avg_loss / count)


In [7]:
gnn = VanillaGNN(dataset.num_classes, 64, 1)
print(gnn)

gnn.fit(train_loader, epochs=25)

VanillaGNN(
  (gnn1): VanillaGNNLayer(
    (linear): Linear(in_features=4, out_features=64, bias=False)
  )
  (gnn2): VanillaGNNLayer(
    (linear): Linear(in_features=64, out_features=1, bias=False)
  )
)
tensor([[[0., 0., 1., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.]],

        [[0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.]],

        [[0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.]],

        [[0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.]],

        [[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.]],

        [[0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [1., 0., 0., 0.],
         [0., 1., 0., 0.]],

        [[0., 0., 0., 1.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.]],

        [[1., 0., 0., 0.

In [8]:
torch.set_printoptions(profile="full")

total_matched = 0

for batch in test_loader:
    x = batch[0]
    y = batch[1]
    predicted = gnn(x, adjacency)
    predicted = predicted.argmax(dim=1)
    matched = (predicted == y).sum().item()
    total_matched += matched
    # print("y", y, "predicted", predicted, "Matched", matched)

print("Total matched", total_matched, "| Accuracy", round(total_matched/len(test_dataset), 4) * 100, "%")

Total matched 6 | Accuracy 75.0 %
